# Dot-And-Boxes Analysis (1x2 Game)
- By: Mehmet Yilmaz
- Date: December 2020

### Class For Printing Clean Looking Output Games

In [1]:
from pandas import *
import numpy as np

# class used for printing dot&boxes in a clean formate in the terminal
class Gride:
    def __init__(self, row, col):
        row = 3 if row*3 <= 3 else row*3-(row-1)
        col = 3 if col*3 <= 3 else col*3-(col-1)
        self.row = row
        self.col = col
        self.grid = np.zeros(shape=(row,col), dtype=str) # create matrix for game
        self.ds = "◼" # text symbol used for displaying games
        self.createGrid()

    # create dots and boxes game
    def createGrid(self):
        for i in range(len(self.grid)):
            for j in range(len(self.grid[i])):
                if(i % 2 == 0):
                    if(j % 2 == 0):
                        self.grid[i][j] = self.ds
                    else:
                        self.grid[i][j] = self.ds
                else:
                    if(j % 2 == 0):
                        self.grid[i][j] = self.ds
                    else:
                        self.grid[i][j] = " "

    # prints dots and boxes game/board
    def printGrid(self, dataFrame=True):
        matrix = self.grid
        if(dataFrame):
            print(DataFrame(matrix))
        else:
            print('\n'.join([' '.join([str(cell) for cell in row]) for row in matrix]))

    # draw player's vertical move on the grid/game
    def makeVrtWall(self, y1, y2, x, player_value):
        p = 1
        if(y1 > y2):
            p = -1
        while(y1 != y2+p):
            self.grid[y1][x] = str(player_value)
            y1 = y1 + (1 * p)
    
    # draw player's horizontal move on the grid/game
    def makeHozWall(self, x1, x2, y, player_value):
        p = 1
        if(x1 > x2):
            p = -1
        while(x1 != x2+p):
            self.grid[y][x1] = str(player_value)
            x1 = x1 + (1 * p)


### Analyzing Game States

In [2]:
# print 1x2 dots & boxes board game using the Grid() class
def print_game_1x2(game):
    game_drawing = Gride(1, 2)
    
    drawing_patterns = {
    0: ["V", "0", "2", "0"],
    1: ["H", "0", "2", "0"],
    2: ["H", "0", "2", "2"],
    3: ["V", "0", "2", "2"],
    4: ["V", "0", "2", "2"],
    5: ["H", "2", "4", "0"],
    6: ["H", "2", "4", "2"],
    7: ["V", "0", "2", "4"],
    }
    
    for i in range(len(game)):
        temp = drawing_patterns[i]
        if(game[i] != "-"):
            if(temp[0] == "V"):
                game_drawing.makeVrtWall(int(temp[1]), 
                                         int(temp[2]), 
                                         int(temp[3]), 
                                         str(game[i]))
            if(temp[0] == "H"):
                game_drawing.makeHozWall(int(temp[1]), 
                                         int(temp[2]), 
                                         int(temp[3]), 
                                         str(game[i]))
    game_drawing.printGrid()

# determines if a player has conquared a sqaure and mark it on the game
def is_conquared(game, player):
    player_1_ID = "1"
    player_2_ID = "2"
    
    current_player = str(player)
    opposite_plyer = str(opposite_player(player))

    sqaure_one = game[:4]
    sqaure_two = game[3:]
    
    conquared_id = "X"

    one_won = [player_1_ID + conquared_id] * (len(game) - 4)
    two_won = [player_2_ID + conquared_id] * (len(game) - 4)
    
    same_player = False
    
    if "-" not in sqaure_one:
        if game[:3] != one_won and game[:3] != two_won:
            game[:3] = [player + conquared_id] * (len(game) - 4)
            same_player = True

    if "-" not in sqaure_two:
        if game[4:] != one_won and game[4:] != two_won:
            game[4:] = [player + conquared_id] * (len(game) - 4)
            same_player = True
    
    return game, same_player

# change next game's player to the next player
def opposite_player(player):
    player_1_ID = "1"
    player_2_ID = "2"
    if(player == player_1_ID):
        return player_2_ID
    else:
        return player_1_ID

# main function
def one_by_two___game_states_data(state, pyr, print_results=True):
    
    # find every possible game state for a 1x2 dots&boxes game
    final = [] # all the states are stored in "final" list
    def get_states(state, player):
        final.append(state)
        if "-" not in state:
            return
        else:
            for i in range(len(state)):
                if state[i] == "-":
                    new_state = state.copy()

                    new_state[i] = str(player)

                    new_state, same_player = is_conquared(new_state, player)

                    next_player = opposite_player(player)

                    if(same_player == True):
                        next_player = player

                    get_states(new_state, next_player)

    get_states(state, pyr) # get every states and store it in final list

    # only get the games were the game has been completed (no more open moves)
    end_game = []
    for i in final:
        if "-" not in i:
            end_game.append(i)

    # count how many times player 1 and player 2 won
    p1_winners = 0
    p2_winners = 0
    for i in end_game:
        if "1X" not in i:
            p2_winners = p2_winners + 1
        if "2X" not in i:
            p1_winners = p1_winners + 1
    
    # save results
    results = [len(final), p1_winners, p2_winners, len(end_game)-p1_winners-p2_winners]
    
    # print results if desired
    if(print_results):
        print("INPUT")
        print_game_1x2(state)
        print("State:  ", np.array(state))
        print("Starter:", pyr)
        print("Num of States:", results[0])
        print("Player 1 Wins:", results[1])
        print("Player 2 Wins:", results[2])
        print("Num of Draws: ", results[3])
    
    return results


### Results:

In [3]:
# main function callings:
'''                Game Layout
                    ◼ 1 ◼ 4 ◼
                    0   3   6
                    ◼ 2 ◼ 5 ◼
'''

# initial state of a 1x2 game
#         0    1    2    3    4    5    6
state = ['-', '-', '-', '-', '-', '-', '-']

# starting player
player = "1"   # only possible IDs: 1 or 2

# get/show results
values = one_by_two___game_states_data(state, player)

INPUT
   0  1  2  3  4
0  ◼  ◼  ◼  ◼  ◼
1  ◼     ◼     ◼
2  ◼  ◼  ◼  ◼  ◼
State:   ['-' '-' '-' '-' '-' '-' '-']
Starter: 1
Num of States: 13700
Player 1 Wins: 720
Player 2 Wins: 3168
Num of Draws:  1152
